In [1]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
pd.set_option('display.width', None)        # Let the display adjust to the window

# 2. Set the display width (optional but often helpful)
#    'None' tries to detect terminal width. 
#    A large number (e.g., 1000) ensures no wrapping unless absolutely necessary.
pd.set_option('display.width', 1000) 

In [2]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")

date_str: 2025-04-07


In [3]:
import sys
from pathlib import Path

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

# Verify path
print(f"Python will look in these locations:\n{sys.path}")


# --- Execute the processor ---
import utils

SOURCE_PATH_OHLCV = f'..\data\{date_str}_df_OHLCV_clean.parquet'
SOURCE_PATH = f'..\data\{date_str}_df_finviz_merged.parquet'
PARQUET_PATH_COVARIANCE = f'..\data\{date_str}_df_cov_emv_matrix.parquet'
PARQUET_PATH_CORRELATION = f'..\data\{date_str}_df_corr_emv_matrix.parquet'

Python will look in these locations:
['C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\python311.zip', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\DLLs', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\Lib', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv', '', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\Pythonwin', 'C:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src']


In [4]:
import pandas as pd

df_ohlcv = pd.read_parquet(SOURCE_PATH_OHLCV)
df_finviz = pd.read_parquet(SOURCE_PATH)

print(f"Loaded df_ohlcv shape: {df_ohlcv.shape}")
print(f"Loaded df_finviz shape: {df_finviz.shape}")


Loaded df_ohlcv shape: (385536, 9)
Loaded df_finviz shape: (1536, 55)


In [5]:
# Combine indices from both dataframes and remove duplicates
tickers = list(set(df_finviz.index))
print(f"Total unique tickers: {len(tickers)}")

Total unique tickers: 1536


In [6]:
df_close = df_ohlcv['Adj Close'].unstack(level=0)

# Assume `tickers` is your predefined list of tickers
common_tickers = df_close.columns.intersection(tickers)
df_close = df_close[common_tickers]

# Assuming `tickers` is your original list of tickers
missing_tickers = list(set(tickers) - set(df_close.columns))

# Print the missing tickers (if any)
if missing_tickers:
    print(f"{len(missing_tickers)} tickers not found in df_close:")
    print(missing_tickers)
else:
    print("All tickers in your list are present in df_close.")

print(f"\nShape of filtered df_close: {df_close.shape}")
display(df_close.head())

All tickers in your list are present in df_close.

Shape of filtered df_close: (251, 1536)


Symbol           A     AA    AAL   AAON    AAPL    ABBV    ABEV    ABNB     ABT   ACGL    ACI   ACIW    ACM     ACN    ACWI    ACWV   ACWX    ADBE    ADC     ADI    ADM     ADP    ADSK   ADT    AEE   AEG    AEM    AEP    AER    AES     AFG    AFL   AFRM    AGCO    AGG    AGI  AGNC    AIG     AIT     AIZ     AJG    AKAM   ALAB     ALB    ALC    ALGN    ALK     ALL    ALLE   ALLY    ALNY   ALSN     ALV     AM    AMAT  AMCR     AMD     AME    AMGN    AMH   AMLP     AMP     AMT    AMX    AMZN      AN   ANET   ANGL    ANSS     AON    AOS    APA     APD    APG    APH     APO    APP    APPF   APTV     AR   ARCC     ARE    ARES    ARGX   ARKB   ARKK     ARM   ARMK     AS    ASML    ASND     ASR  ASTS    ASX    ATI     ATO     ATR     AU   AUR     AVB   AVDE   AVDV   AVEM    AVGO   AVLV   AVTR   AVUS   AVUV     AVY     AWI     AWK    AXON     AXP    AXS   AXTA     AYI   AZEK    AZN      AZO      BA   BABA    BAC     BAH   BALL    BAM     BAP    BAX   BBAX   BBCA     BBD   BBEU   BBIN   BBIO  \
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
2024-04-08  143.39  35.93  13.90  92.53  167.66  163.83  2.3873  160.27  108.31  89.91  20.42  33.00  96.52  327.74  107.60  101.54  51.84  484.28  54.71  193.48  60.88  241.45  245.89  6.36  71.04  5.94  60.50  81.14  86.32  17.48  124.73  83.41  34.11  117.22  92.72  14.82  8.37  76.21  195.48  178.53  242.14  105.82  69.92  129.36  83.56  320.06  42.58  168.96  131.34  37.41  156.04  81.67  118.83  13.37  207.29  8.72  169.90  179.74  261.24  35.50  44.13  430.18  182.05  18.53  185.19  159.03  74.35  27.01  344.10  317.83  85.49  33.79  231.75  39.57  57.79  114.46  78.05  233.20  76.07  29.97  18.77  120.59  131.67  386.71  71.79  48.10  129.25  32.18  14.52  976.59  148.88  318.56  2.27  10.63  51.79  113.47  138.89  22.88  3.20  180.34  61.75  63.67  57.19  132.00  62.84  25.67  87.53  90.55  214.51  120.28  117.30  311.21  221.60  64.02  32.96  267.25  50.17  66.34  3076.09  181.56  70.18  36.60  145.15  65.75  40.01  170.32  41.15  46.40  65.27  2.6392  56.88  57.94  28.27   
2024-04-09  146.31  36.41  13.95  89.24  168.87  164.16  2.4270  162.66  110.19  86.74  20.39  33.83  94.96  330.36  107.76  101.72  51.96  492.55  55.97  200.79  61.57  244.75  247.93  6.45  71.67  5.91  61.55  81.92  85.34  17.47  122.43  81.48  35.37  119.69  93.04  15.02  8.44  73.93  194.41  176.11  242.11  107.57  68.37  126.88  82.41  330.25  43.01  165.01  132.35  37.97  155.90  81.11  119.09  13.36  208.65  8.93  170.78  180.00  262.36  35.50  43.78  424.08  186.49  18.39  185.67  160.27  73.69  27.10  344.50  317.39  85.00  33.65  233.73  39.51  56.98  110.33  76.52  232.49  78.93  30.23  18.75  123.34  130.34  384.14  68.98  48.62  126.81  31.86  14.94  983.67  153.67  315.06  2.31  10.79  51.74  113.77  138.69  22.81  3.25  182.94  61.79  63.70  57.61  131.80  62.74  25.86  87.58  90.62  215.81  118.93  119.09  311.27  217.38  62.29  33.54  265.67  49.83  67.03  3043.62  178.12  71.44  36.82  141.39  66.49  40.59  167.49  41.72  46.72  65.56  2.6482  56.74  57.85  28.43   


In [7]:
df_close_sorted = df_close.sort_index()
print(f'df_close_sorted shape: {df_close_sorted.shape}')
display(df_close_sorted.head())

# Calculate daily returns
df_close_returns = df_close_sorted.pct_change().dropna()
print(f'df_close_returns shape: {df_close_returns.shape}')
display(df_close_returns.head())


df_close_sorted shape: (251, 1536)


Symbol           A     AA    AAL   AAON    AAPL    ABBV    ABEV    ABNB     ABT   ACGL    ACI   ACIW    ACM     ACN    ACWI    ACWV   ACWX    ADBE    ADC     ADI    ADM     ADP    ADSK   ADT    AEE   AEG    AEM    AEP    AER    AES     AFG    AFL   AFRM    AGCO    AGG    AGI  AGNC    AIG     AIT     AIZ     AJG    AKAM   ALAB     ALB    ALC    ALGN    ALK     ALL    ALLE   ALLY    ALNY   ALSN     ALV     AM    AMAT  AMCR     AMD     AME    AMGN    AMH   AMLP     AMP     AMT    AMX    AMZN      AN   ANET   ANGL    ANSS     AON    AOS    APA     APD    APG    APH     APO    APP    APPF   APTV     AR   ARCC     ARE    ARES    ARGX   ARKB   ARKK     ARM   ARMK     AS    ASML    ASND     ASR  ASTS    ASX    ATI     ATO     ATR     AU   AUR     AVB   AVDE   AVDV   AVEM    AVGO   AVLV   AVTR   AVUS   AVUV     AVY     AWI     AWK    AXON     AXP    AXS   AXTA     AYI   AZEK    AZN      AZO      BA   BABA    BAC     BAH   BALL    BAM     BAP    BAX   BBAX   BBCA     BBD   BBEU   BBIN   BBIO  \
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
2024-04-08  143.39  35.93  13.90  92.53  167.66  163.83  2.3873  160.27  108.31  89.91  20.42  33.00  96.52  327.74  107.60  101.54  51.84  484.28  54.71  193.48  60.88  241.45  245.89  6.36  71.04  5.94  60.50  81.14  86.32  17.48  124.73  83.41  34.11  117.22  92.72  14.82  8.37  76.21  195.48  178.53  242.14  105.82  69.92  129.36  83.56  320.06  42.58  168.96  131.34  37.41  156.04  81.67  118.83  13.37  207.29  8.72  169.90  179.74  261.24  35.50  44.13  430.18  182.05  18.53  185.19  159.03  74.35  27.01  344.10  317.83  85.49  33.79  231.75  39.57  57.79  114.46  78.05  233.20  76.07  29.97  18.77  120.59  131.67  386.71  71.79  48.10  129.25  32.18  14.52  976.59  148.88  318.56  2.27  10.63  51.79  113.47  138.89  22.88  3.20  180.34  61.75  63.67  57.19  132.00  62.84  25.67  87.53  90.55  214.51  120.28  117.30  311.21  221.60  64.02  32.96  267.25  50.17  66.34  3076.09  181.56  70.18  36.60  145.15  65.75  40.01  170.32  41.15  46.40  65.27  2.6392  56.88  57.94  28.27   
2024-04-09  146.31  36.41  13.95  89.24  168.87  164.16  2.4270  162.66  110.19  86.74  20.39  33.83  94.96  330.36  107.76  101.72  51.96  492.55  55.97  200.79  61.57  244.75  247.93  6.45  71.67  5.91  61.55  81.92  85.34  17.47  122.43  81.48  35.37  119.69  93.04  15.02  8.44  73.93  194.41  176.11  242.11  107.57  68.37  126.88  82.41  330.25  43.01  165.01  132.35  37.97  155.90  81.11  119.09  13.36  208.65  8.93  170.78  180.00  262.36  35.50  43.78  424.08  186.49  18.39  185.67  160.27  73.69  27.10  344.50  317.39  85.00  33.65  233.73  39.51  56.98  110.33  76.52  232.49  78.93  30.23  18.75  123.34  130.34  384.14  68.98  48.62  126.81  31.86  14.94  983.67  153.67  315.06  2.31  10.79  51.74  113.77  138.69  22.81  3.25  182.94  61.79  63.70  57.61  131.80  62.74  25.86  87.58  90.62  215.81  118.93  119.09  311.27  217.38  62.29  33.54  265.67  49.83  67.03  3043.62  178.12  71.44  36.82  141.39  66.49  40.59  167.49  41.72  46.72  65.56  2.6482  56.74  57.85  28.43   


df_close_returns shape: (250, 1536)


Symbol             A        AA       AAL      AAON      AAPL      ABBV      ABEV      ABNB       ABT      ACGL       ACI      ACIW       ACM       ACN      ACWI      ACWV      ACWX      ADBE       ADC       ADI       ADM       ADP      ADSK       ADT       AEE       AEG       AEM       AEP       AER       AES       AFG       AFL      AFRM      AGCO       AGG       AGI      AGNC       AIG       AIT       AIZ       AJG      AKAM      ALAB       ALB       ALC      ALGN       ALK       ALL      ALLE      ALLY      ALNY      ALSN       ALV        AM      AMAT      AMCR       AMD       AME      AMGN       AMH      AMLP       AMP       AMT       AMX      AMZN        AN      ANET      ANGL      ANSS       AON       AOS       APA       APD       APG       APH       APO       APP      APPF      APTV        AR      ARCC       ARE      ARES      ARGX      ARKB      ARKK       ARM      ARMK        AS      ASML      ASND       ASR      ASTS       ASX       ATI       ATO       ATR        AU  \
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
2024-04-09  0.020364  0.013359  0.003597 -0.035556  0.007217  0.002014  0.016630  0.014912  0.017358 -0.035257 -0.001469  0.025152 -0.016162  0.007994  0.001487  0.001773  0.002315  0.017077  0.023031  0.037782  0.011334  0.013667  0.008296  0.014151  0.008868 -0.005051  0.017355  0.009613 -0.011353 -0.000572 -0.018440 -0.023139  0.036939  0.021071  0.003451  0.013495  0.008363 -0.029917 -0.005474 -0.013555 -0.000124  0.016538 -0.022168 -0.019171 -0.013763  0.031838  0.010099 -0.023378  0.007690  0.014969 -0.000897 -0.006857  0.002188 -0.000748  0.006561  0.024083  0.005180  0.001447  0.004287  0.000000 -0.007931 -0.014180  0.024389 -0.007555  0.002592  0.007797 -0.008877  0.003332  0.001162 -0.001384 -0.005732 -0.004143  0.008544 -0.001516 -0.014016 -0.036082 -0.019603 -0.003045  0.037597  0.008675 -0.001066  0.022805 -0.010101 -0.006646 -0.039142  0.010811 -0.018878 -0.009944  0.028926  0.007250  0.032174 -0.010987  0.017621  0.015052 -0.000965  0.002644 -0.001440 -0.003059   
2024-04-10 -0.021940 -0.017303 -0.038710 -0.019162 -0.011133 -0.006518 -0.028595 -0.014632 -0.011253  0.010606 -0.003433 -0.022761 -0.012110 -0.026305 -0.011414 -0.009831 -0.013472 -0.010821 -0.033053 -0.038199 -0.000162 -0.016874 -0.035978 -0.026357 -0.023162 -0.011844 -0.015272 -0.020996 -0.011483 -0.033200 -0.013069 -0.003068 -0.050891  0.011697 -0.012038  0.006658 -0.047393 -0.002570 -0.017540 -0.007325 -0.016852 -0.029655  0.043001 -0.011743 -0.005946 -0.007358 -0.024413  0.008424 -0.035436 -0.032131 -0.022386  0.003329 -0.021748 -0.002994 -0.005512 -0.027996 -0.021314 -0.007889 -0.014446 -0.013521 -0.007081 -0.008418 -0.056250 -0.023382  0.001508 -0.036439 -0.022120 -0.008118 -0.014543 -0.010649 -0.010941  0.004458 -0.015659 -0.015692 -0.001931  0.020937 -0.002352 -0.030496 -0.050424  0.008601 -0.006933 -0.052781  0.001841 -0.003540  0.016961 -0.023241 -0.012854 -0.009730 -0.008032 -0.015381 -0.007093 -0.011014 -0.073593 -0.013902 -0.032277 -0.013360 -0.006489 -0.018413   
2024-04-11  0.005800  0.

In [8]:
cov_emv_matrix, corr_emv_matrix = utils.get_cov_corr_ewm_matrices_chunked(df_close_returns, span=21, return_corr=True, return_cov=True)

print(f'cov_emv_matrix shape: {cov_emv_matrix.shape}')
display(cov_emv_matrix.head())
print(f'corr_emv_matrix shape: {corr_emv_matrix.shape}')
display(corr_emv_matrix.head())  

cov_emv_matrix shape: (1536, 1536)


Symbol         A        AA       AAL      AAON      AAPL      ABBV      ABEV      ABNB       ABT      ACGL       ACI      ACIW       ACM       ACN      ACWI      ACWV      ACWX      ADBE       ADC       ADI       ADM       ADP      ADSK       ADT       AEE       AEG       AEM       AEP       AER       AES       AFG       AFL      AFRM      AGCO       AGG       AGI      AGNC       AIG       AIT       AIZ       AJG      AKAM      ALAB       ALB       ALC      ALGN       ALK       ALL      ALLE      ALLY      ALNY      ALSN       ALV        AM      AMAT      AMCR       AMD       AME      AMGN       AMH      AMLP       AMP       AMT       AMX      AMZN        AN      ANET      ANGL      ANSS       AON       AOS       APA       APD       APG       APH       APO       APP      APPF      APTV        AR      ARCC       ARE      ARES      ARGX      ARKB      ARKK       ARM      ARMK        AS      ASML      ASND       ASR      ASTS       ASX       ATI       ATO       ATR        AU       AUR  \
Symbol                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
A       0.000536  0.000776  0.000517  0.000599  0.000568  0.000336  0.000090  0.000514  0.000231  0.000358  0.000080  0.000450  0.000296  0.000411  0.000385  0.000185  0.000295  0.000380  0.000056  0.000776  0.000346  0.000274  0.000357  0.000221  0.000183  0.000540  0.000331  0.000085  0.000581  0.000525  0.000250  0.000347  0.001101  0.000506 -0.000048  0.000331  0.000187  0.000425  0.000440  0.000387  0.000263  0.000395  0.000793  0.000737  0.000403  0.000341  0.000546  0.000291  0.000263  0.000463  0.000638  0.000509  0.000423  0.000441  0.000639  0.000191  0.000647  0.000476  0.000148  0.000221  0.000371  0.000625 -0.000047  0.000079  0.000542  0.000286  0.000899  0.000136  0.000390  0.000153  0.000224  0.001088  0.000401  0.000459  0.000605  0.000992  0.001031  0.000267  0.000502  0.000857  0.000390  0.000396  0.001052  0.000301  0.000117  0.000674  0.000810  0.000390  0.000654  0.000409  0.000320  0.000229  0.000771  0.000560  0.000849  0.000179  0.000267  0.000425  0.001086   
AA      0.000776  0.001654  0.000843  0.000978  0.001033  0.000436  0.000186  0.000954  0.000233  0.000570  0.000108  0.000922  0.000524  0.000676  0.000666  0.000280  0.000525  0.000713  0.000066  0.001210  0.000456  0.000410  0.000699  0.000504  0.000240  0.000966  0.000457  0.000035  0.001001  0.000890  0.000405  0.000551  0.002042  0.000971 -0.000064  0.000509  0.000311  0.000641  0.000907  0.000709  0.000382  0.000693  0.001532  0.001336  0.000521  0.000673  0.001076  0.000504  0.000419  0.000927  0.000848  0.000981  0.000830  0.000761  0.001063  0.000333  0.001254  0.000828  0.000108  0.000372  0.000672  0.001092 -0.000190  0.000080  0.000985  0.000603  0.001529  0.000239  0.000678  0.000255  0.000410  0.002015  0.000681  0.000819  0.001077  0.001812  0.002124  0.000478  0.000983  0.001485  0.000664  0.000704  0.001882  0.000389  0.000469  0.001228  0.001351  0.000701  0.001335  0.000781  0.000561  0.000464  0.001001  0.001091  0.001518  0.000241  0.000425  0.000764  0.001871   


corr_emv_matrix shape: (1536, 1536)


Symbol         A        AA       AAL      AAON      AAPL      ABBV      ABEV      ABNB       ABT      ACGL       ACI      ACIW       ACM       ACN      ACWI      ACWV      ACWX      ADBE       ADC       ADI       ADM       ADP      ADSK       ADT       AEE       AEG       AEM       AEP       AER       AES       AFG       AFL      AFRM      AGCO       AGG       AGI      AGNC       AIG       AIT       AIZ       AJG      AKAM      ALAB       ALB       ALC      ALGN       ALK       ALL      ALLE      ALLY      ALNY      ALSN       ALV        AM      AMAT      AMCR       AMD       AME      AMGN       AMH      AMLP       AMP       AMT       AMX      AMZN        AN      ANET      ANGL      ANSS       AON       AOS       APA       APD       APG       APH       APO       APP      APPF      APTV        AR      ARCC       ARE      ARES      ARGX      ARKB      ARKK       ARM      ARMK        AS      ASML      ASND       ASR      ASTS       ASX       ATI       ATO       ATR        AU       AUR  \
Symbol                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
A       1.000000  0.823539  0.625731  0.718568  0.757969  0.653081  0.172952  0.735153  0.527770  0.574730  0.165548  0.684551  0.716864  0.752207  0.859864  0.543075  0.688548  0.614105  0.155253  0.881669  0.552762  0.605550  0.762403  0.454472  0.433247  0.764038  0.560013  0.203899  0.845091  0.695423  0.484767  0.596852  0.725645  0.631292 -0.455539  0.506667  0.505283  0.692941  0.671429  0.648588  0.502559  0.611722  0.639154  0.806234  0.696427  0.626702  0.683853  0.461366  0.635847  0.620495  0.662795  0.757354  0.716070  0.696515  0.830895  0.485659  0.754704  0.882251  0.372012  0.488703  0.735617  0.876688 -0.097668  0.154262  0.764656  0.517570  0.807308  0.762386  0.771365  0.419296  0.640754  0.835682  0.795761  0.800354  0.799300  0.863045  0.542189  0.584702  0.736994  0.807530  0.713490  0.697576  0.836116  0.540616  0.142440  0.785798  0.809098  0.668099  0.651545  0.707617  0.492261  0.371502  0.574011  0.751840  0.853510  0.481049  0.561833  0.585607  0.701997   
AA      0.823539  1.000000  0.580745  0.668061  0.785662  0.482344  0.204129  0.776276  0.302408  0.520768  0.128086  0.798645  0.723572  0.704712  0.846973  0.469162  0.697283  0.655890  0.104790  0.782855  0.414765  0.516469  0.849756  0.590004  0.323410  0.778349  0.440658  0.047766  0.829136  0.671492  0.446855  0.539422  0.766435  0.690322 -0.343607  0.443088  0.477384  0.595199  0.786957  0.677070  0.414992  0.610673  0.703033  0.831968  0.512543  0.703620  0.766988  0.455630  0.577301  0.707509  0.501724  0.830745  0.800388  0.683348  0.786378  0.481223  0.833442  0.874270  0.154699  0.467546  0.759340  0.871665 -0.225026  0.088523  0.791105  0.621496  0.781913  0.760586  0.763226  0.398039  0.668222  0.881192  0.769856  0.813616  0.810495  0.898154  0.635805  0.595844  0.822453  0.796920  0.692232  0.705560  0.851942  0.397960  0.326105  0.814351  0.768621  0.684067  0.756995  0.768776  0.491707  0.428504  0.424110  0.833616  0.869138  0.368389  0.509724  0.599606  0.688964   


In [9]:
cov_emv_matrix.to_parquet(PARQUET_PATH_COVARIANCE)
print(f'cov_emv_matrix parqueted to {PARQUET_PATH_COVARIANCE}')
display(cov_emv_matrix.info())

corr_emv_matrix.to_parquet(PARQUET_PATH_CORRELATION)
print(f'corr_emv_matrix parqueted to {PARQUET_PATH_CORRELATION}')
display(corr_emv_matrix.info())

cov_emv_matrix parqueted to ..\data\2025-04-07_df_cov_emv_matrix.parquet
<class 'pandas.core.frame.DataFrame'>
Index: 1536 entries, A to ZTS
Columns: 1536 entries, A to ZTS
dtypes: float64(1536)
memory usage: 18.0+ MB


None

corr_emv_matrix parqueted to ..\data\2025-04-07_df_corr_emv_matrix.parquet
<class 'pandas.core.frame.DataFrame'>
Index: 1536 entries, A to ZTS
Columns: 1536 entries, A to ZTS
dtypes: float64(1536)
memory usage: 18.0+ MB


None